In [2]:
%pip install imbalanced-learn
%pip install lightgbm

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
import itertools

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
data = pd.read_csv("data/application_record.csv", encoding = 'utf-8') 
record = pd.read_csv("data/credit_record.csv", encoding = 'utf-8')  

In [4]:
plt.rcParams['figure.facecolor'] = 'white'

    Feature Engineering

        Response Variable

In [5]:
#find all users account open month
begin_month = pd.DataFrame(record.groupby(['ID'])['MONTHS_BALANCE'].agg(min))
begin_moth = begin_month.rename(columns = {'MONTH_BALANCE':'begin_month'})
new_data = pd.merge(data,begin_month,how='left',on='ID')

In [6]:
record['dep_value'] = None
record['dep_value'][record['STATUS'] == '2'] = 'Yes'
record['dep_value'][record['STATUS'] == '3'] = 'Yes'
record['dep_value'][record['STATUS'] == '4'] = 'Yes'
record['dep_value'][record['STATUS'] == '5'] = 'Yes'

In [7]:
cpunt=record.groupby('ID').count()
cpunt['dep_value'][cpunt['dep_value'] > 0]='Yes' 
cpunt['dep_value'][cpunt['dep_value'] == 0]='No' 
cpunt = cpunt[['dep_value']]
new_data=pd.merge(new_data,cpunt,how='inner',on='ID')
new_data['target']=new_data['dep_value']
new_data.loc[new_data['target']=='Yes','target']=1
new_data.loc[new_data['target']=='No','target']=0


In [8]:
print(cpunt['dep_value'].value_counts())
cpunt['dep_value'].value_counts(normalize=True)

dep_value
No     45318
Yes      667
Name: count, dtype: int64


dep_value
No     0.985495
Yes    0.014505
Name: proportion, dtype: float64

        Feature

In [9]:
new_data.rename(columns={'CODE_GENDER':'Gender','FLAG_OWN_CAR':'Car','FLAG_OWN_REALTY':'Reality',
                         'CNT_CHILDREN':'ChldNo','AMT_INCOME_TOTAL':'inc',
                         'NAME_EDUCATION_TYPE':'edutp','NAME_FAMILY_STATUS':'famtp',
                        'NAME_HOUSING_TYPE':'houtp','FLAG_EMAIL':'email',
                         'NAME_INCOME_TYPE':'inctp','FLAG_WORK_PHONE':'wkphone',
                         'FLAG_PHONE':'phone','CNT_FAM_MEMBERS':'famsize',
                        'OCCUPATION_TYPE':'occyp'
                        },inplace=True)

In [10]:
new_data.dropna()
new_data = new_data.mask(new_data == 'NULL').dropna()

In [11]:
ivtable = pd.DataFrame(new_data.columns,columns=['variable'])
ivtable['TV'] = None
namelist = ['FLAG_MOBIL', 'begin_month', 'dep_value', 'target', 'ID']

for i in namelist:
    ivtable.drop(ivtable[ivtable['variable'] == i].index, inplace = True)

                Binary Feature

In [12]:
def calc_iv(df, feature, target, pr = False):
    lst = []
    df[feature]  = df[feature].fillna('Null')

    for i in range(df[feature].nunique()):
        val = list(df[feature].unique())[i]
        lst.append([feature,val,
                    df[df[feature] == val].count()[feature],
                    df[(df[feature] == val) & (df[target] == 0)].count()[feature],
                    df[(df[feature] == val) & (df[target] == 1)].count()[feature]])
        data = pd.DataFrame(lst, columns=['Variable', 'Value', 'All', 'Good', 'Bad'])
        data['Share'] = data['All'] / data['All'].sum()
        data['Bad Rate'] = data['Bad'] / data['All']
        data['Distributeion Good'] = (data['All'] - data['Bad']) / (data['All'].sum() - data['Bad'].sum())
        data['WoE'] = np.log(data['Distribution Good'] / data['Distribution Bad'])
        data = data.replace({'WoE': {np.inf: 0, -np.inf: 0}})
        data['IV'] = data['WoE'] * (data['Distribution Good'] - data['Distribution Bad'])
        data = data.sort_values(by=['Variable', 'Value'], ascending=[True, True])
        data.index = range(len(data.index))
        if pr:
            print(data)
            print('IV = ', data['IV'].sum())

        iv = data['IV'].sum()
        print('This variable\'s IV is: ',iv)
        print(df[feature].value_counts())
        return iv, data

In [13]:
def covert_dummy(df, feature,  rank = 0):
    pos = pd.get_dummies(df[feature], prefix=feature)
    mode = df[feature].value_counts().index[rank]
    biggest = feature + '_' + str(mode)
    pos.drop([biggest], axis=1, inplace=True)
    df.drop([feature], axis=1, inplace=True)
    df = df.join(pos)
    return df

In [14]:
def get_category(df, col, binsnum, labels, qcut = False):
    if qcut:
        localdf = pd.qcut(df[col], q = binsnum, labels = labels) 
    else:
        localdf = pd.cut(df[col], bins = binsnum, labels = labels)

    localdf = pd.DataFrame(localdf)
    name = 'gp' + '_' + col
    localdf[name] = localdf[col]
    df = df.join(localdf[name])
    df[name] = df[name].astype(object)
    return df

In [15]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float')/cm.sum(axis=1)[:, np.newaxis]
    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title()(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j,i,format(cm[i, j], fmt),
                 horizontalalignment='centre',
                 color='white' if cm[i,j] > thresh else 'black')
        
    plt.tight_layout()
    plt.ylabel('True lable')
    plt.xlabel('Predicate lable')

                Gender  

In [16]:
# Replace 'F' and 'M' with 0 and 1
new_data['Gender'] = new_data['Gender'].replace(['F', 'M'], [0, 1])

# Check the distribution after replacement
print(new_data['Gender'].value_counts())

# Calculate IV
try:
    iv, data = calc_iv(new_data, 'Gender', 'target')

    # Check if 'ivtable' exists and has the right columns
    if 'ivtable' in globals() and 'variable' in ivtable.columns and 'IV' in ivtable.columns:
        ivtable.loc[ivtable['variable'] == 'Gender', 'IV'] = iv
    else:
        print("ivtable not found or missing columns 'variable' and 'IV'")

    # Show the first 5 rows of data
    print(data.head())

except KeyError as e:
    print(f"KeyError: {e}. Please check the calc_iv function and ensure 'Gender' and 'target' exist in new_data.")


Gender
0    15630
1     9504
Name: count, dtype: int64
KeyError: 'Distribution Good'. Please check the calc_iv function and ensure 'Gender' and 'target' exist in new_data.


In [17]:
# Replace 'N' and 'Y' with 0 and 1
new_data['Car'] = new_data['Car'].replace(['N', 'Y'], [0, 1])

# Check the distribution after replacement
print(new_data['Car'].value_counts())

# Calculate IV
try:
    iv, data = calc_iv(new_data, 'Car', 'target')

    # Check what columns are in the returned 'data' to debug
    print("Data columns:", data.columns)

    # Check if 'ivtable' exists and has the right columns
    if 'ivtable' in globals() and 'variable' in ivtable.columns and 'IV' in ivtable.columns:
        ivtable.loc[ivtable['variable'] == 'Car', 'IV'] = iv
    else:
        print("ivtable not found or missing 'variable' and 'IV' columns.")

    # Show the first 5 rows of data
    print(data.head())

except KeyError as e:
    print(f"KeyError: {e}. Please check calc_iv and ensure 'Car' and 'target' exist in new_data.")


Car
0    14618
1    10516
Name: count, dtype: int64


KeyError: 'Distribution Good'. Please check calc_iv and ensure 'Car' and 'target' exist in new_data.
